<a href="https://colab.research.google.com/github/dtht2d/bispectrum_component/blob/main/bispectrum/example/test_S04_calc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bispectrum - S(O)4 Class Function

In [ ]:
%autosave 30

Autosaving every 30 seconds


In [ ]:
!pip install biopython

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 28.5 MB/s eta 0:00:00


In [ ]:
!pip install numpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install ipython-autotime
%load_ext autotime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 18.1 MB/s eta 0:00:00
time: 400 µs (started: 2023-05-03 03:13:13 +00:00)


# **Bispectrum- S04 Class Function Testing**

## Data preparation

In [ ]:
def neighbor_params(center_atom_id:int, r_mu, R_cut, input_file:str, output_dir:str, file_type:str):
    """
    This function is used to get input value from cif file and define neighbor atoms parameters 
    """
    if file_type == "cif":
        # DATA PREPARATION
        dico = MMCIF2Dict(input_file)
        df_cif = pd.DataFrame.from_dict(dico, orient='index')
        x = df_cif.iloc[-3]
        y = df_cif.iloc[-2]
        z = df_cif.iloc[-1]
        atom_type = df_cif.iloc[-4]
        x_array = np.array(x[0], dtype=float)
        y_array = np.array(y[0], dtype=float)
        z_array = np.array(z[0], dtype=float)
        atom_type_array = np.array(atom_type[0], dtype=str)
        df = pd.DataFrame({"atom_type": atom_type_array, "X": x_array, "Y": y_array, "Z": z_array})
        # Estimate list of potentially atoms in the center cell
        id = center_atom_id
        # id
        x_i = df['X'].iloc[id]
        y_i = df['Y'].iloc[id]
        z_i = df['Z'].iloc[id]
        # print(x_i,y_i,z_i)
        X_array = df['X'].to_numpy()
        Y_array = df['Y'].to_numpy()
        Z_array = df['Z'].to_numpy()
        X_k_array = X_array - x_i
        Y_k_array = Y_array - y_i
        Z_k_array = Z_array - z_i
        r_ik = np.sqrt(np.square(X_k_array) + np.square(Y_k_array) + np.square(Z_k_array))
        df['X_k'], df['Y_k'], df['Z_k'], df['r_ik'] = X_k_array, Y_k_array, Z_k_array, r_ik
        # INPUT values
        cell_length = df.iloc[4]  # index row start from 0 _cell_length_a at row 5 index [4]
        #r_mu = 0.0779  # scale atomic radius w.r.t cell length
        #R_cut = 0.25  # scaled value w.r.t cell length (for Si-Si case)
        condition = (df['r_ik'] + r_mu <= R_cut) & (df['r_ik'] != 0)
        df_ik = df.loc[condition].copy(deep=True)
        # ANGEL CONVERSION
        # theta_0
        r_ik_array = df_ik['r_ik'].to_numpy()  # r_ik from selected neighbors
        r_0_array = np.full((r_ik_array.shape), R_cut)
        theta_0_array = np.pi * (np.divide(r_ik_array, r_0_array))
        # theta
        Z_k_abs_array = np.abs(df_ik['Z_k'].to_numpy())
        theta_array = np.arccos(np.divide(Z_k_abs_array, r_ik_array))
        # phi
        X_k_array = df_ik['X_k'].to_numpy()
        Y_k_array = df_ik['Y_k'].to_numpy()
        phi_array = np.arctan(np.divide(Y_k_array, X_k_array))
        # convert angle to positive value between [0,2pi]
        phi_array_convert = np.mod(phi_array, 2 * np.pi)
        for angle in phi_array_convert:
            if (angle >= 2 * np.pi) and (angle < 0):
                raise ValueError('phi angle in between 0 and 2pi')
        # replace NaN with 0: (code will have error for invalid value center atom values 0/0)_
        df_ik['theta_0'] = theta_0_array
        df_ik['theta_0'] = df_ik['theta_0'].replace(np.nan, 0)
        df_ik['theta'] = theta_array
        df_ik['theta'] = df_ik['theta'].replace(np.nan, 0)
        df_ik['phi'] = phi_array_convert
        df_ik['phi'] = df_ik['phi'].replace(np.nan, 0)
        # array for weight coefficient w.r.t to atom type
        w_ik_arr = np.full((r_ik_array.shape), 1)
        # delta function delta=1 if i and k has the same element type, if not delta =0
        delta = np.full((r_ik_array.shape), 0)
        delta_arr = np.where(df_ik['atom_type'] == df_ik['atom_type'].iloc[0], 1, delta)
        df_ik['w_ik'] = w_ik_arr
        df_ik['delta'] = delta_arr
    # save the DataFrame as a JSON file
        neighbor_list_path = output_dir + '-atom-' + str(center_atom_id) + '-neighbor-list.json'
        df_ik.to_json(neighbor_list_path, orient='records')
        """
        This function is to read in the neighbor list file
        """
        with open(neighbor_list_path, 'r') as f:
            json_data = json.load(f)

            # create a list of dictionaries from the JSON data
            data_list = [dict(row) for row in json_data]

            # extract data from the list of dictionaries
            r_ik_array = np.array([float(row['r_ik']) for row in data_list])
            theta_0_array = np.array([float(row['theta_0']) for row in data_list])
            theta_array = np.array([float(row['theta']) for row in data_list])
            phi_array = np.array([float(row['phi']) for row in data_list])
            w_ik_array = np.array([float(row['w_ik']) for row in data_list])
            delta_array = np.array([float(row['delta']) for row in data_list])
            r_cut_array = np.full((r_ik_array.shape), R_cut)
            # return a dictionary with the extracted data
    else:
        raise ValueError(f"Unsupported file type: {file_type}")
    return {
            'r_ik': r_ik_array,
            'theta_0': theta_0_array,
            'theta': theta_array,
            'phi': phi_array,
            'w_ik': w_ik_array,
            'delta': delta_array,
            'r_cut': r_cut_array
        }

time: 3.64 ms (started: 2023-05-03 03:13:13 +00:00)


**Example**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
time: 1min 15s (started: 2023-05-03 03:13:13 +00:00)


In [ ]:
import numpy as np
import pandas as pd
from Bio.PDB.MMCIF2Dict import MMCIF2Dict
import json
center_atom_id = 17
r_mu =0.0779 
R_cut= 0.25
input_file = "/content/drive/MyDrive/bispectrump_components/data/avgBL-Model.cif"
output_dir =  "/content/drive/MyDrive/bispectrump_components/data"
file_type = 'cif'
input_data = neighbor_params(center_atom_id, r_mu, R_cut, input_file, output_dir, file_type='cif')

print (input_data)
# Convert dictionary to DataFrame
df = pd.DataFrame(input_data)
# Display DataFrame
print(df)

{'r_ik': array([0.16062009, 0.12295675, 0.12219716, 0.12619947, 0.11925231]), 'theta_0': array([2.01841164, 1.54512013, 1.53557477, 1.58586934, 1.49856872]), 'theta': array([0.69919773, 0.3820468 , 1.18488014, 1.25583529, 1.15492866]), 'phi': array([0.049713  , 1.25332279, 5.41038219, 1.13378905, 6.18181825]), 'w_ik': array([1., 1., 1., 1., 1.]), 'delta': array([1., 1., 1., 1., 1.]), 'r_cut': array([0.25, 0.25, 0.25, 0.25, 0.25])}
       r_ik   theta_0     theta       phi  w_ik  delta  r_cut
0  0.160620  2.018412  0.699198  0.049713   1.0    1.0   0.25
1  0.122957  1.545120  0.382047  1.253323   1.0    1.0   0.25
2  0.122197  1.535575  1.184880  5.410382   1.0    1.0   0.25
3  0.126199  1.585869  1.255835  1.133789   1.0    1.0   0.25
4  0.119252  1.498569  1.154929  6.181818   1.0    1.0   0.25
time: 1.06 s (started: 2023-05-03 03:14:28 +00:00)


# Methods

## **Conditions check**
---

### **Clebsch Gordan calculation condition**

$$C^{jm}_{{j_1}{m_1}{j_2}{m_2}} C^{jm'}_{{j_1}{m'_1}{j_2}{m'_2}} \equiv H^{jmm'}_{{{j_1}{m_1}{m'_1}} ,{{j_2}{m_2}{m'_2}}}$$


Ref[12]. Ch3.12, eq(3.171)

$$
\begin{align*} C_{j_1m_1j_2m_2}^{jm} &= \ \delta(m_1 +m_2,m) \\ & \times \left[\frac{(2j+1)(j+j_1-j_2)!(j-j_1+j_2)!(j_1+j_2-j)!}{(j+j_1+j_2+1)!} \right]^{\frac{1}{2}} \\ & \times \left[ \frac{(j+m)!(j-m)!}{(j_1+m_1)!(j_1-m_1)!(j_2+m_2)!(j_2-m_2)!}\right]^{\frac{1}{2}} \\ & \times \displaystyle\sum_s\frac{(-1)^{j_2+m_2+s}(j_2+j+m_1-s)!(j_1-m_1+s)!}{s!(j-j_1+j_2-s)!(j+m-s)!(j_1-j_2-m+s)!} \end{align*}
$$
Note: $s$ values already taken into considerration to ensure factorial is non-negative integer in Clebsh Gordan class function calculation 
In accordance with vector addition rules $j_1+j_2=j$, unless the triangular conditions (triangular in equalities) are fulfilled

**General conditions for input $(j,j_1,j_2)$**

(1) $|j_1-j_2|\ \le j \le j_1+j_2$ and $j_1+j_2+j$  are non-negative integer and $j_1 +j_2=j$

(2) $(j+j_1-j_2)$ and $(j-j_1+j_2)$ and ($j_1+j_2-j$) and is non-negative integer

(3) $j_1,j_2,j$ not exceed a positive integer  $J= j_1+j_2+j$

$j= 0, \frac{1}{2},1,...\infty$  and $m, m' = -j,-j+1, ...,j-1,j$

**Condition for set $(j_1,j_2,j,m_1,m_2,m)$**

(4) $m_1,m_2,m$ are integer or half-integer (positive or negative) numbers

(5) $j_1 +m_1, j_1-m_1, \ j_2 +m_2, j_2-m_2 \ j +m, j -m $ are non-negative integer

(6)$m_1+m_2=m$ 

(7)$|m_1| \le j_1, \ |m_2| \le j_2,\ |m| \le j$ 

(8)$j_2+j+m_1$ is positive integer and $j_1-j_2-m$ is integer

**Condition for set $(j_1,j_2,j,m'_1,m'_2,m')$**

(4') $m'_1,m'_2,m'$ are integer or half-integer (positive or negative) numbers

(5') $j_1 +m'_1, j_1- m'_1, \ j_2 +m'_2, j_2-m'_2 \ j +m', \ j - m' $ are non negative integer

(6')$m'_1+m'_2=m'$

(7')$|m'_1| \le j_1, \ |m'_2| \le j_2,\ |m'| \le j$ 

(8')$j_2+j+m'_1$ and $j_1-j_2-m'$ is integer


### **Wigner D calculation condition**

\begin{align*}d^j_{mm'}{(\theta)}= [(j+m)!(j-m)!(j+m')!(j-m')!]^{\frac{1}{2}} \\ \times \displaystyle\sum_k(-1)^k\frac{(cos\frac{\theta}{2})^{2j-2k+m-m'}(sin\frac{\theta}{2})^{2k-m+m'}}{k!(j+m-k)!(j-m'-k)!(m'-m+k)!)} \end{align*}
Note: $k$ values already taken into consideration to ensure factorial is non-negative integer

**Conditions that have included in Clebsh Gordan coefficient**

 $|m| \leq j, \ |m'| \leq j, \ j+m, \ j-m , \ j+m', \ j-m'$ are non-negative integer

**Conditions need to add**

(9) $m'-m \geq 0, \ m_1' -m_1 \geq 0, \ m'_2-m_2 $ non-negative integer








In [4]:
import numpy as np

def generate_m_values(j, j1, j2):
    """
    This function generates (m1, m2, m, m1p, m2p, mp) from input set (j1,j2,j)
    and only keeps sets that satisfy the conditions.
    """
    J=j1 + j2 + j
    #Condition 1
    if not ((abs(j1 - j2) <= j <= j1 + j2) or isinstance(J,int) or (J >=0) or (j1 + j2) == j):
        raise ValueError("Condition (1) is not satisfied.")
    #Condition 2
    if not ((isinstance(val,int) or val >=0 for val in [j + j1 - j2,j - j1 + j2,j1 + j2 - j])):
        raise ValueError("(𝑗+𝑗1−𝑗2) and (𝑗−𝑗1+𝑗2) and (𝑗1+𝑗2−𝑗) and is non-negative integer")
    #Condition 3
    if j1 > J or j2 > J or j > J:
        raise ValueError("𝑗1,𝑗2,𝑗 not exceed a positive integer 𝐽=𝑗1+𝑗2+𝑗")
    # Generate m values
    m1_vals = np.linspace(-j1, j1, int(2 * j1 + 1))
    m2_vals = np.linspace(-j2, j2, int(2 * j2 + 1))
    m_vals = np.linspace(-j, j, int(2 * j + 1))
    mp_vals = m_vals.copy()
    m1p_vals = m1_vals.copy()
    m2p_vals = m2_vals.copy()
    m1, m2, m, m1p, m2p, mp = np.meshgrid(m1_vals, m2_vals, m_vals, m1p_vals, m2p_vals, mp_vals)
    s = np.stack((m1.ravel(), m2.ravel(), m.ravel(), m1p.ravel(), m2p.ravel(), mp.ravel()), axis=1)
    full_list = s
    keep_list = []
    for i in range(len(s)):
      m1_val, m2_val, m_val, m1p_val, m2p_val, mp_val = s[i]
      #Condition 4-8: Clebsch-Gordan calc for set (𝑗1,𝑗2,𝑗,𝑚1,𝑚2,𝑚), (𝑗1,𝑗2,𝑗,𝑚1p,𝑚2p,𝑚p)
      c4 = [isinstance(val, (int, float)) and val % 0.5 == 0 for val in [m1_val, m2_val, m_val, m1p_val, m2p_val, mp_val]]
      c5 = [isinstance(vals, int) and vals >= 0 for vals in [j1 + m1_val, j1 - m1_val, j2 + m2_val, j2 - m2_val, j + m_val, j - m_val]]
      c5_p = [isinstance(vals, int) and vals >= 0 for vals in [j1 + m1p_val, j1 - m1p_val, j2 + m2p_val, j2 - m2p_val, j + mp_val, j - mp_val]]
      c6 = [m1_val+m2_val == m_val and m1p_val+m2p_val == mp_val]
      c7 = [abs(val) <= limit for val, limit in [(m1_val, j1), (m2_val, j2), (m_val, j), (m1p_val, j1), (m2p_val, j2), (mp_val, j)]]
      c8 = [j2 + j + m1_val >= 0 and j1 - j2 - m_val >= 0 and isinstance(j2 + j + m1_val, int) and isinstance(j1 - j2 - m_val, int)]
      c8_p = [j2 + j + m1p_val >= 0 and j1 - j2 - mp_val >= 0 and isinstance(j2 + j + m1p_val, int) and isinstance(j1 - j2 - mp_val, int)]
      #Condition 9: Wigner-D calc 
      c9 = [isinstance(vals, int) and vals >= 0 for vals in [mp_val-m_val, m1p_val-m1_val, m2p_val-m2_val]]
    if (c4) and (c5 and c5_p) and (c6) and (c7) and (c8 and c8_p) and (c9):
      keep_list.append(s[i])
    else:
      pass
    return keep_list, full_list

**Example**

In [12]:
#test set 1
keep_list_1,s1= generate_m_values(3,1/2,5/2)
print ("full list from set 1", s1.shape)
print ("set_1:",keep_list_1, "keep_list #", np.array(keep_list_1).shape)

#test set 2
keep_list_2,s = generate_m_values(3,2,1)
print ("full list from set 2", s1.shape, "keep_list #", np.array(keep_list_2).shape)
print ("set_2:", keep_list_2)


full list from set 1 (7056, 6)
set_1: [array([0.5, 2.5, 3. , 0.5, 2.5, 3. ])] keep_list # (1, 6)
full list from set 2 (7056, 6) keep_list # (1, 6)
set_2: [array([2., 1., 3., 2., 1., 3.])]


**Comment:** the probability amplitude that 𝑗1 and 𝑗2 are coupled into a resultant angular momentum 𝑗 with projection 𝑚.
From input $(j, j1, j2)$ and the code function shows that there one and only unique set $(m,m_1,m_2, m, m_1', m_2')$ which makes sense 

Because: conditions check are uses for calculate 
- Clebsch Gordan coefficient represents **the probability** amplitude that $j_1$ and $j_2$ are coupled into a resultant angular momentum $j$ with projection $m$, and $m_1', \ m_2', \ m':$ are Eigenvalues w.r.t $j_1, \ j_2, \ j$ along rotated 
- Wigner D:  describes the rotation of quantum states under the action of a rotation operator

**Conclusion**: 
For a given input of $(j, j_1, j_2)$ there is only one set of magnetic quantum numbers that describe the system's angular momentum. These numbers are represented by $(m, m_1,m_2$ respectively. when we rotate the system, the magnetic quantum numbers change. We denote the new magnetic quantum numbers as $(m, m_1, m_2)$ which correspond to the rotated system. These numbers describe the new projections or orientations of the angular momentum after the rotation.
The code result show that there is a unique set of magnetic quantum numbers (m, m_1, m_2) that describe the system's angular momentum. Under rotation, this set of magnetic quantum numbers changes to a new set $(m', m_1', m_2')$ that represents the rotated system's angular momentum projections or orientations. 


# **Compute Bispectrum Function**

In [ ]:
import numpy as np
import json
import pandas as pd
from Bio.PDB.MMCIF2Dict import MMCIF2Dict
from typing import Dict

fact_cache = {}
def fact(n):
    if n < 0 or not np.isclose(n, int(n)):
        raise ValueError("Invalid input parameter: n must be a non-negative integer.")
    if n in fact_cache:
        return fact_cache[n]
    result = np.math.factorial(int(n))
    fact_cache[n] = result
    return result
class Bispectrum:
    """
    Calculate bispectrum- S(0)4 components
    """
    def __init__(self,j, j1, j2, params: Dict[str, np.ndarray]):
        '''
            j: j index
            j1: j1 index
            j2: j2 index
            input_data: input data dictionary with extracted values:
            r_ik (array): dictance from center atom to n neighbor atom, dim = [n,]
            theta_0 (array): first angle of rotation [0, pi] , dim = [n,]
            theta (array): second angle of rotation [0, pi], dim = [n,]
            phi (array): third angle of rotation [0, 2pi], dim = [n,]
            w_ik (array): weight coefficient, dim = [n,]
            delta (array): delta function, dim = [n,]
            r_cut (array): cutoff distance, dim = [n,]
        '''
        self.j=j
        self.j1= j1
        self.j2=j2
        self.params = params
        w_ik_array = self.params['w_ik']
        delta_array = self.params['delta']
        r_ik_array = self.params['r_ik']
        r_cut_array = self.params['r_cut']
        theta_0_array = self.params['theta_0']
        theta_array = self.params['theta']
        phi_array = self.params['phi']
        #Condition check
        if not (abs(j1 - j2) <= j <= j1 + j2 and j1 + j2 - j % 1 != 0.5):
            raise ValueError("Invalid input parameters: j1, j2, j must satisfy the triangle inequality.\ "
                             "j1+j2-j must not be a half-integer")
        J = (j1 + j2 + j)
        if J < (int(j1 + j2 + j)) and J < 0:
            raise ValueError("Invalid input parameters: j1, j2, j must not exceed a positive integer J")

    @staticmethod
    def generate_m_values(j, j1, j2):
        """
        This function generates (m1, m2, m, m1p, m2p, mp) from input set (j1,j2,j)
        and only keeps sets that satisfy the conditions.
        Return: a one and only unique set (m1, m2, m, m1p, m2p, mp)
        """
        J = j1 + j2 + j
        # Condition 1
        if not ((abs(j1 - j2) <= j <= j1 + j2) or isinstance(J, int) or (J >= 0) or (j1 + j2) == j):
            raise ValueError("Condition (1) is not satisfied.")
        # Condition 2
        if not ((isinstance(val, int) or val >= 0 for val in [j + j1 - j2, j - j1 + j2, j1 + j2 - j])):
            raise ValueError("(𝑗+𝑗1−𝑗2) and (𝑗−𝑗1+𝑗2) and (𝑗1+𝑗2−𝑗) and is non-negative integer")
        # Condition 3
        if j1 > J or j2 > J or j > J:
            raise ValueError("𝑗1,𝑗2,𝑗 not exceed a positive integer 𝐽=𝑗1+𝑗2+𝑗")
        # Generate m values
        m1_vals = np.linspace(-j1, j1, int(2 * j1 + 1))
        m2_vals = np.linspace(-j2, j2, int(2 * j2 + 1))
        m_vals = np.linspace(-j, j, int(2 * j + 1))
        mp_vals = m_vals.copy()
        m1p_vals = m1_vals.copy()
        m2p_vals = m2_vals.copy()
        m1, m2, m, m1p, m2p, mp = np.meshgrid(m1_vals, m2_vals, m_vals, m1p_vals, m2p_vals, mp_vals)
        s = np.stack((m1.ravel(), m2.ravel(), m.ravel(), m1p.ravel(), m2p.ravel(), mp.ravel()), axis=1)
        keep_list = []
        full_list = []
        for i in range(len(s)):
            full_list.append(s[i])
            m1_val, m2_val, m_val, m1p_val, m2p_val, mp_val = s[i]
            # Condition 4-8: Clebsch-Gordan calc for set (𝑗1,𝑗2,𝑗,𝑚1,𝑚2,𝑚), (𝑗1,𝑗2,𝑗,𝑚1p,𝑚2p,𝑚p)
            c4 = [isinstance(val, (int, float)) and val % 0.5 == 0 for val in
                  [m1_val, m2_val, m_val, m1p_val, m2p_val, mp_val]]
            c5 = [isinstance(vals, int) and vals >= 0 for vals in
                  [j1 + m1_val, j1 - m1_val, j2 + m2_val, j2 - m2_val, j + m_val, j - m_val]]
            c5_p = [isinstance(vals, int) and vals >= 0 for vals in
                    [j1 + m1p_val, j1 - m1p_val, j2 + m2p_val, j2 - m2p_val, j + mp_val, j - mp_val]]
            c6 = [m1_val + m2_val == m_val and m1p_val + m2p_val == mp_val]
            c7 = [abs(val) <= limit for val, limit in
                  [(m1_val, j1), (m2_val, j2), (m_val, j), (m1p_val, j1), (m2p_val, j2), (mp_val, j)]]
            c8 = [j2 + j + m1_val >= 0 and j1 - j2 - m_val >= 0 and isinstance(j2 + j + m1_val, int) and isinstance(
                j1 - j2 - m_val, int)]
            c8_p = [
                j2 + j + m1p_val >= 0 and j1 - j2 - mp_val >= 0 and isinstance(j2 + j + m1p_val, int) and isinstance(
                    j1 - j2 - mp_val, int)]
            # Condition 9: Wigner-D calc
            c9 = [isinstance(vals, int) and vals >= 0 for vals in [mp_val - m_val, m1p_val - m1_val, m2p_val - m2_val]]
        if (c4) and (c5 and c5_p) and (c6) and (c7) and (c8 and c8_p) and (c9):
            keep_list.append(s[i])
        else:
            pass
        return keep_list, full_list
    #Clebsch-Gordan Coefficient
    @staticmethod
    def clebsch_gordan(j1, j2, j, m1, m2, m):
        """
        Definition:
            A Clebsch-Gordan coefficients are vector addition coefficients. They play an important role in decomposition of
            reducible representations of rotation. Let j1 and j2 with projections on m1 and m2 on the quantization axis.
            The coefficients represent the probability amplitude that j1 and j2 are coupled into a resultant angular momentum
            j with projection m.
        Args:
            j, j1, j2 (scalar): angular momentum
            m, m1, m2 (scalar): eigenvalue of angular momentum j, j1, j2 respectively
            mp, mp1, mp2 (scalar): eigenvalue of j, j1, j2 along rotated axis respectively
        Returns: Clebsch-Gordan coefficients, real number
        ==========================Reference==================================
        [5] Chapter 8 D.A. Varshalovich, A.N. Moskalev, V.K Khersonskii,
                Quantum Theory of Angular Momentum (1988)
        [12] Chapter 3 Biedenharn, L., & Louck, J.D. ,
                Encyclopedia of Mathematics and its Applications (1981)
        """
        if m1 + m2 != m:
            return 0.0  # delta function fails
        prefactor = np.sqrt((2 * j + 1) * fact(j + j1 - j2) * fact(j - j1 + j2) * fact(j1 + j2 - j) / fact(j + j1 + j2 + 1))
        coefficient = np.sqrt(fact(j + m) * fact(j - m) / (fact(j1 + m1) * fact(j1 - m1) * fact(j2 + m2) * fact(j2 - m2)))
        sum = 0.0
        smin = max(0, int(m1 - j1), int(j2 - j1 + m))
        smax = min(int(j2 + j + m1), int(j - j1 + j2), int(j +m))
        for s in range(smin, smax + 1):
            den = fact(s) * fact(j - j1 + j2 - s) * fact(j + m - s) * fact(j1 - j2 - m + s)
            num = ((-1) ** (j2 + m2 + s)) * fact(j2 + j + m1 - s) * fact(j1 - m1 + s)
            sum += num / den
        cg = prefactor * coefficient * sum
        return cg
    #Coupling Coefficient
    @classmethod
    def H(cls, j1, j2, j, m1, m2, m, m1p, m2p, mp):
        CG = cls.clebsch_gordan(j1, j2, j, m1, m2, m)
        CGp = cls.clebsch_gordan(j1, j2, j, m1p, m2p, mp)
        H_coeff = CG * CGp
        return H_coeff
    @staticmethod
    def compute_dsmall(j, m, mp, theta):
        """
        This method is used to calculate the Wigner d small- real function involving trigonometric functions
        Returns: Wigner d - real function
        ==========================Reference==================================
        [5] Chapter 4.3.1-(p.76,eq.4)  D.A. Varshalovich, A.N. Moskalev, V.K Khersonskii,
        """
        kmax = max(0,m - mp)
        kmin = min(j + m, j - mp)
        term1 = np.sqrt(fact(j + m) * fact(j - m) * fact(j + mp) * fact(j - mp))
        sum = 0
        for k in range(int(kmax), int(kmin) + 1):
            numerator = (-1) ** k * (np.cos(theta / 2)) ** (2 * j - 2 * k + m - mp) * \
                        (np.sin(theta / 2)) ** (2 * k - m + mp)
            denominator = fact(k) * fact(j + m - k) * fact(j - mp - k) * fact(mp - m + k)
            sum += numerator / denominator
        return sum*term1
    @classmethod
    def wigner_D(cls, j, m, mp, theta_0, theta, phi):
        """
        This method is used to calculate the Wigner D matrix
        Args:
            theta_0 (scalar): first angle of rotation [0, pi]
            theta (scalar): second angle of rotation [0, pi]
            phi (scalar): third angle of rotation [0, 2*pi]
        Returns: complex number, Wigner D function
        ==========================Reference==================================
        [5] Chapter 4.3-(p.76,eq.1)  D.A. Varshalovich, A.N. Moskalev, V.K Khersonskii,
        """
        term1 = np.cos(m *theta_0) - 1j*(np.sin(m * theta_0))
        term2 = cls.compute_dsmall(j, m, mp, theta)
        term3 = np.cos(mp * phi) -1j*(np.sin(mp * phi))
        result = term1 * term2 * term3
        return result
    @classmethod
    def U_rot(cls, j, m, mp, theta_0, theta, phi):
        """
        This method is used to calculate the rotation matrix U
        Returns: complex number, Rotational matrix U function
        ==========================Reference==================================
        [5] Chapter 4  D.A. Varshalovich, A.N. Moskalev, V.K Khersonskii,
                  Quantum Theory of Angular Momentum (1988)
        """
        mpp_vals = np.linspace(-j, j, int(2 * j + 1))
        U = 0
        for mpp in mpp_vals:
            term1 = cls.wigner_D(j, m, mpp, phi, theta, -phi)
            term2 = np.cos(mpp * theta_0) - 1j * (np.sin(mpp * theta_0))
            term3 = cls.wigner_D(j, mpp, mp, phi, -theta, -phi)
            Um_mp = term1 * term2 * term3
            U += Um_mp
        return U
    @classmethod
    def u_small(cls, j, m, mp, params):
        """
        Args:
            j (scalar): angular momentum
            m (scalar): eigenvalue of angular momentum
            mp (scalar): eigenvalue of j along rotated axis
            params (dict): a dictionary containing the following keys, its values:
                - w_ik (array): the coefficients that are dimensionless weights that are chosen to distinguish atoms
                  of different types, while the central atom is arbitrarily assigned a unit weight, dimensin (1,k)
                - delta (array): the Dirac delta function, indicates only neighbor atom of element the same as center atom
                  contribute to partial density,  dimension (1,k)
                - r_ik (array): distance from center atom to neighbor atom, dimension (1,k), k is number of neighbor atoms
                  in cutoff radius, array exclude center atom as well
                - r_cut (array): cutoff radius
                - theta_0: array for theta_0 angel (fist angle of rotation [0,pi])
                  of neighbor atoms in reference frame of center atom, dimension (k+1,)
                - theta: array for theta angel ( second angle of rotation [0,pi])
                  of neighbor atoms in reference frame of center atom, dimension (k+1,)
                - phi: array for phi angel (third angle of rotation [0,2pi])
                  of neighbor atoms in reference frame of center atom, dimension (k+1,)
        Returns: expansion coefficients density function u_jm_mp
        """
        w_ik_array = params['w_ik']
        delta_array = params['delta']
        r_ik_array = params['r_ik']
        r_cut_array = params['r_cut']
        theta_0_array = params['theta_0']
        theta_array = params['theta']
        phi_array = params['phi']

        # Calculate cutoff_function
        f_cut_arr = (1 / 2) * (np.cos(np.pi * (np.divide(r_ik_array, r_cut_array))) + 1)
        # Calculate rotational matrix U for all k=n neighbor atoms
        U_ik_array = np.array([cls.U_rot(j, m, mp, theta_0, theta, phi) for theta_0, theta, phi in
                               zip(theta_0_array, theta_array, phi_array)], dtype='complex')
        # Compute u_jmmp
        u_jmmp = np.dot((f_cut_arr * U_ik_array), (w_ik_array * delta_array))
        return u_jmmp
    @classmethod
    def evaluate(cls, j, j1, j2, params):
        m_list, full_list = cls.generate_m_values(j,j1,j2)
        B_total = 0
        for i in m_list:
          m1, m2, m, m1p, m2p, mp = i[0], i[1], i[2], i[3], i[4], i[5]
          H_coeff = cls.H(j1, j2, j, m1, m2, m, m1p, m2p, mp)
          u_jmmp = cls.u_small(j, m, mp, params)
          u1_j1m1m1p = cls.u_small(j1, m1, m1p, params)
          u2_j2m2m2p = cls.u_small(j2, m2, m2p, params)
          B = np.conj(u_jmmp) * (H_coeff * u1_j1m1m1p * u2_j2m2m2p)
          B_total += B
        return B_total


time: 5.39 ms (started: 2023-05-03 03:34:53 +00:00)


Example

In [ ]:
from sympy import *
import timeit
from sympy.physics.quantum.cg import CG
#INPUT
center_atom_id = 17
r_mu =0.0779 
R_cut= 0.25
input_file = "/content/drive/MyDrive/bispectrump_components/data/avgBL-Model.cif"
output_dir =  "/content/drive/MyDrive/bispectrump_components/data"
file_type = 'cif'
input_data = neighbor_params(center_atom_id, r_mu, R_cut, input_file, output_dir, file_type='cif')

time: 27.5 ms (started: 2023-05-03 03:34:57 +00:00)


## Test functions inside Bispectrum class

In [ ]:
#possible list of m values
B = Bispectrum(j=2, j1=1/2, j2=3/2, params=input_data)

time: 608 µs (started: 2023-05-03 03:34:59 +00:00)


In [ ]:
#test Clebsch Gordan Coefficient calculation
CG_coeff = B.clebsch_gordan(j1, j2, j, m1, m2, m)
print(CG_coeff)
#Compare sympy
cg = CG(j1,m1,j2,m2,j,m)
cg = cg.doit()

#test Coupling coefficient calculation
H_coeff = B.H(j1, j2, j, m1, m2, m, m1p, m2p, mp)
print(H_coeff)

#Test Wigner D
j, m, mp, theta_0, theta,phi= 1, -1, 0, np.pi, np.pi/2, 0
WD = B.wigner_D(j, m, mp, theta_0, theta,phi)
#Test U_rot
U = B.U_rot(j, m, mp, theta_0, theta,phi)
print (WD)
print(U)
#Test u-small
u = B.u_small(j, m, mp, params=input_data)

1.0000000000000002
1.0000000000000004
(-0.7071067811865476+8.659560562354934e-17j)
(2.7755575615628914e-16-8.659560562354933e-17j)
time: 26.2 ms (started: 2023-05-03 03:35:01 +00:00)


### Rotational matrix $U^j_{mm'}$
Ref[1], p.30, eq.(A.4), the 4D hysperspherical harmonics satisfy the following symmetry sign-inversion relation
$$
U^j_{mm'}=(-1)^{m'-m}(U^j_{-m,-m'})^*
$$

In [54]:
#Test U_rot
j, m, mp, theta_0, theta,phi= 1, -1, 0, np.pi, np.pi/2, 0
U = B.U_rot(j, m, mp, theta_0, theta,phi)
U_p = B.U_rot(j, -m, -mp, theta_0, theta,phi)
U_test =(-1)**(mp-m)*np.conj(U_p)
print(U_test)
print (U)

(2.7755575615628914e-16-8.659560562354933e-17j)
(2.7755575615628914e-16-8.659560562354933e-17j)
time: 5.49 ms (started: 2023-05-03 03:50:02 +00:00)


Test B calculation:
 

In [ ]:
#Test bispectrum calculation
#possible list of m values
B_1 = Bispectrum(j=2, j1=1/2, j2=3/2, params=input_data)
B1 = B_1.evaluate(j=2, j1=1/2, j2=3/2, params=input_data)

B_2 = Bispectrum(j=2, j1=3/2, j2=1/2, params=input_data)
B2 = B_2.evaluate(j=2, j1=3/2, j2=1/2, params=input_data)
print(B1)
print(B2)


(1.3631640353765455+0.17853639602096893j)
(1.3631640353765455+0.17853639602096893j)
time: 195 ms (started: 2023-05-03 03:36:10 +00:00)


Comment: When interchange $j_1, and j_2$ B are the same